In [4]:
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers import InputExample
import pandas as pd, math
from torch.utils.data import DataLoader
import torch

num_epochs = 1
train_batch_size = 5
model_save_path = "./model_dump"
model_name = 'vectara/hallucination_evaluation_model' # base model, use 'vectara/hallucination_evaluation_model' if you want to further fine-tune ours

model = CrossEncoder(model_name, num_labels=1, automodel_args={'ignore_mismatched_sizes':True})

In [5]:
# Load some training examples as such, using a pandas dataframe with source and summary columns:
train_examples, test_examples = [], []
# load train

# # trial dev
# train = pd.read_excel('../data/trial_v1.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['label'] == "Hallucination" else 1)))
# train = pd.read_excel('../data/dev_aware.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['label'] == "Hallucination" else 1)))
# train = pd.read_excel('../data/dev_agnostic.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['label'] == "Hallucination" else 1)))

# # solar
# train = pd.read_excel('../data/train_aware.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['solar'] == "Hallucination" else 1)))
# train = pd.read_excel('../data/train_agnostic.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['solar'] == "Hallucination" else 1)))

# # baseline
# train = pd.read_excel('../data/train_aware.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['baseline'] == "Hallucination" else 1)))
# train = pd.read_excel('../data/train_agnostic.xlsx')
# for i, row in train.iterrows():
#     train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['baseline'] == "Hallucination" else 1)))

# special filtered results for solar
json = pd.read_json('../data/filter_results.json')
index = 1 # 0, 1, 2
train_aware_filter = json[index]["aware"]
train_agnostic_filter = json[index]["agnostic"]

train = pd.read_excel('../data/train_aware.xlsx')
for i, row in train.iterrows():
    if train_aware_filter[i] == True:
        train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['solar'] == "Hallucination" else 1)))
train = pd.read_excel('../data/train_agnostic.xlsx')
for i, row in train.iterrows():
    if train_agnostic_filter[i] == True:
        train_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['solar'] == "Hallucination" else 1)))

# load test
test = pd.read_excel('../data/test_aware.xlsx')
for i, row in test.iterrows():
    test_examples.append(InputExample(texts=[str(row['src'])+" "+str(row["tgt"]), str(row['hyp'])], label=int(0 if row['label'] == "Hallucination" else 1)))

print("Train examples:", len(train_examples), "Test examples:", len(test_examples))
print(train_examples[0])
print(test_examples[0])

Train examples: 6084 Test examples: 1500
<InputExample> label: 1, texts: `` i would have written to him , but a few words from you will go further than all the apologetical sesquipedalities i could muster on the occasion . What is the meaning of apologetical ? apologetic; expressing or expressing regret or apology
<InputExample> label: 1, texts: Sadly , at the end of his life , Luther became somewhat of a nutso . Crazy person ; crackpot or lunatic .; A crazy person .


In [6]:
# Then train the model as such as per the Cross Encoder API:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
model.fit(train_dataloader=train_dataloader,
          evaluator=CEBinaryClassificationEvaluator.from_input_examples(test_examples, name='test'),
          epochs=num_epochs,
          evaluation_steps=10_000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          show_progress_bar=True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1217 [00:00<?, ?it/s]